# MeterReader - The Trainer

This notebok implements the YOLOv11 based trainers for the models used by the meterreader

- Frames : Get Extract the Frame inside of the meter from the original image
- Counter : Extract the Counter from within the frame
- Digits: Get the value of each digit of the Counter

## Training Data

The training data is taken from the respective Roboflow projects. The data is annotated using either Roboflow or manually via script (Digits)

## Models

The resulting models are saved in the same directory as the downloaded roboflow datasets 





## Citations and Acknowledgements

@software{yolo11_ultralytics,
  author = {Glenn Jocher and Jing Qiu},
  title = {Ultralytics YOLO11},
  version = {11.0.0},
  year = {2024},
  url = {https://github.com/ultralytics/ultralytics},
  orcid = {0000-0001-5950-6979, 0000-0002-7603-6750, 0000-0003-3783-7069},
  license = {AGPL-3.0}
}

In [17]:
# Uses the Ultralytics YOLOv11 Libraries
import ultralytics
ultralytics.checks()

from ultralytics import YOLO
import torch

# Datasets are stored, annotated, on roboflow
from roboflow import Roboflow

# Load custom configurations. Variables accessible with config.get('<<VARIABLE_NAME>>')
from config import ConfigLoader

import os, shutil

Ultralytics 8.3.49 🚀 Python-3.12.8 torch-2.5.1 CPU (Apple M4)
Setup complete ✅ (10 CPUs, 24.0 GB RAM, 237.5/926.4 GB disk)


In [18]:
# Load the config variables
config = ConfigLoader()

print("The loaded configuration:")

config.print_config()

# Check to see if running on a Mac or not (Apple Silicon)
device_to_use = config.get('DEVICE')

if torch.backends.mps.is_available():
    device_to_use = 'mps'

print(f"Device used for training: {device_to_use}")


Loaded configuration from /Users/yonz/Workspace/meterreader2/config.yaml
The loaded configuration:
DEFAULT_CONFIDENCE: 66
DEFAULT_DEBUG: false
DEFAULT_DISPLAY_IMAGE: false
DEFAULT_FILEPATH: ./static/
DEFAULT_MAX_HEIGHT: 1280
DEFAULT_MERGE_PARALLEL: true
DEFAULT_OVERLAP: 10
DEVICE: cpu
EXPAND_BBOX_BY: 1.03
INCREASE_CONFIDENCE_BY: 1.2
RF_API_KEY: IOgzRx9sitOu5Sz6YTOC
RF_PROJECT_NAME_COUNTERS: counter-rbnzk
RF_PROJECT_NAME_DIGITS: meter-digits-itvsi
RF_PROJECT_NAME_FRAME: frame-f0pux
RF_SERVER_URL: http://172.30.32.0:9001/
RF_VERSION_COUNTERS: '1'
RF_VERSION_DIGITS: '3'
RF_VERSION_FRAME: '2'
RF_WORKSPACE_NAME: meterreader
YOLO_WEIGHTS_DIR: ../weights
canny_threshold1: 120
canny_threshold2: 250
digit_max_height: 40
digit_min_height: 20
digit_y_alignment: 10

Device used for training: mps


In [19]:
# Load the roboflow datasets
# Chengae the comments to select the right model

rf = Roboflow(api_key=config.get('RF_API_KEY'))
workspace = rf.workspace(config.get('RF_WORKSPACE_NAME'))

#project = workspace.project(config.get('RF_PROJECT_NAME_DIGITS'))
#project = workspace.project(config.get('RF_PROJECT_NAME_FRAME'))
project = workspace.project(config.get('RF_PROJECT_NAME_COUNTERS'))

# version = project.version(config.get('RF_VERSION_DIGITS'))
# version = project.version(config.get('RF_VERSION_FRAME'))
version = project.version(config.get('RF_VERSION_COUNTERS'))


dataset = version.download("yolov11")
# dataset = version.download("yolov11", location="../datasets")
print(f"Dataset Location: {dataset.location}")


                
                


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to counter-1 in yolov11:: 100%|██████████| 294/294 [00:00<00:00, 14700.72it/s]

Dataset Location: /Users/yonz/Workspace/meterreader2/counter-1


In [20]:
# Train

# Load a model
model = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training)

run_name=os.path.basename(dataset.location)
project_name="meterreader_YOLO"

# Train the model with MPS
results = model.train(data=f"{dataset.location}/data.yaml", cache=True, epochs=50, imgsz=[640,640], device=device_to_use, 
                                                            name=run_name, project=project_name, exist_ok=True)

resulting_model = f"{results.save_dir}/weights/best.pt"

New https://pypi.org/project/ultralytics/8.3.51 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.49 🚀 Python-3.12.8 torch-2.5.1 MPS (Apple M4)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/Users/yonz/Workspace/meterreader2/counter-1/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=[640, 640], save=True, save_period=-1, cache=True, device=mps, workers=8, project=meterreader_YOLO, name=counter-1, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=No

train: Scanning /Users/yonz/Workspace/meterreader2/counter-1/train/labels... 123 images, 0 backgrounds, 0 corrupt: 100%|██████████| 123/123 [00:00<00:00, 3265.89it/s]

train: New cache created: /Users/yonz/Workspace/meterreader2/counter-1/train/labels.cache
WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.



train: Caching images (0.1GB RAM): 100%|██████████| 123/123 [00:00<00:00, 8077.84it/s]
val: Scanning /Users/yonz/Workspace/meterreader2/counter-1/valid/labels... 12 images, 0 backgrounds, 0 corrupt: 100%|██████████| 12/12 [00:00<00:00, 4130.62it/s]

val: New cache created: /Users/yonz/Workspace/meterreader2/counter-1/valid/labels.cache
WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


...

      50/50      5.29G     0.4091     0.5076     0.8168         11        640: 100%|██████████| 8/8 [00:10<00:00,  1.25s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.73s/it]

                   all         12         12      0.996          1      0.995      0.966



50 epochs completed in 0.193 hours.
Optimizer stripped from meterreader_YOLO/counter-1/weights/last.pt, 5.5MB
Optimizer stripped from meterreader_YOLO/counter-1/weights/best.pt, 5.5MB

Validating meterreader_YOLO/counter-1/weights/best.pt...
Ultralytics 8.3.49 🚀 Python-3.12.8 torch-2.5.1 MPS (Apple M4)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:05<00:00,  5.55s/it]


                   all         12         12      0.996          1      0.995      0.966
Speed: 0.4ms preprocess, 173.5ms inference, 0.0ms loss, 159.2ms postprocess per image
Results saved to meterreader_YOLO/counter-1


In [21]:
# Validate the model just rained
print(f"Validate the model using the valid dataset : {resulting_model}")
# Validate the model
results = model.val()  # no arguments needed, dataset and settings remembered

# Access specific metrics
print(results.results_dict)




Validate the model using the valid dataset : meterreader_YOLO/counter-1/weights/best.pt
Ultralytics 8.3.49 🚀 Python-3.12.8 torch-2.5.1 MPS (Apple M4)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning /Users/yonz/Workspace/meterreader2/counter-1/valid/labels.cache... 12 images, 0 backgrounds, 0 corrupt: 100%|██████████| 12/12 [00:00<?, ?it/s]

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.



val: Caching images (0.0GB RAM): 100%|██████████| 12/12 [00:00<00:00, 15679.64it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.97it/s]


                   all         12         12      0.996          1      0.995      0.966
Speed: 0.4ms preprocess, 8.6ms inference, 0.0ms loss, 3.4ms postprocess per image
Results saved to meterreader_YOLO/counter-1
{'metrics/precision(B)': 0.9956707834469842, 'metrics/recall(B)': 1.0, 'metrics/mAP50(B)': 0.995, 'metrics/mAP50-95(B)': 0.96600169985261, 'fitness': 0.968901529867349}


In [22]:
# Evaluate the model on the test data (model is already loaded)
print(f"Predict on the Test dataset from {resulting_model}")

results = model.val(split='test')

# Print the results
#print(results)

# Access specific metrics
# Access specific metrics
print(results.results_dict)

Predict on the Test dataset from meterreader_YOLO/counter-1/weights/best.pt
Ultralytics 8.3.49 🚀 Python-3.12.8 torch-2.5.1 MPS (Apple M4)


val: Scanning /Users/yonz/Workspace/meterreader2/counter-1/test/labels... 6 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6/6 [00:00<00:00, 4007.94it/s]

val: New cache created: /Users/yonz/Workspace/meterreader2/counter-1/test/labels.cache
WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.



val: Caching images (0.0GB RAM): 100%|██████████| 6/6 [00:00<00:00, 12246.14it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:06<00:00,  6.78s/it]


                   all          6          6      0.992          1      0.995      0.965
Speed: 13.0ms preprocess, 870.7ms inference, 0.0ms loss, 164.0ms postprocess per image
Results saved to meterreader_YOLO/counter-1
{'metrics/precision(B)': 0.9922627146239112, 'metrics/recall(B)': 1.0, 'metrics/mAP50(B)': 0.995, 'metrics/mAP50-95(B)': 0.9651082569741398, 'fitness': 0.9680974312767259}


In [23]:
# Export the trained custom model 
# Device = cpu for general purpose prediction (other than on a mac)
# 
if results:
    trained_model = YOLO(resulting_model)
    trained_model.export(device='cpu',  imgsz=[192,800],
                 int8=True, data=f"{dataset.location}/data.yaml",
                 name=run_name, project=project_name, exist_ok=True)


Ultralytics 8.3.49 🚀 Python-3.12.8 torch-2.5.1 CPU (Apple M4)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from 'meterreader_YOLO/counter-1/weights/best.pt' with input shape (1, 3, 192, 800) BCHW and output shape(s) (1, 5, 3150) (5.2 MB)

TorchScript: starting export with torch 2.5.1...
TorchScript: export success ✅ 0.7s, saved as 'meterreader_YOLO/counter-1/weights/best.torchscript' (10.4 MB)

Export complete (0.8s)
Results saved to /Users/yonz/Workspace/meterreader2/meterreader_YOLO/counter-1/weights
Predict:         yolo predict task=detect model=meterreader_YOLO/counter-1/weights/best.torchscript imgsz=192,800 int8 
Validate:        yolo val task=detect model=meterreader_YOLO/counter-1/weights/best.torchscript imgsz=192,800 data=/Users/yonz/Workspace/meterreader2/counter-1/data.yaml int8 WARNING ⚠️ non-PyTorch val requires square images, 'imgsz=[192, 800]' will not work. Use export 'imgsz=800' if val is required.
Visualize:  

In [26]:
# Run some tests...

# Load a model
print(f"Loading the model for testing from {resulting_model}")
digits_model = YOLO(resulting_model)  # load prev. trained model

# Images to predict are all here .... (path depends on the model being trained)
images = f"/Users/yonz/Workspace/images/counter_original/IMG_69*.jpg"

# run prediction on some test images - Uses the CPU, even on a M4 Mac
results = digits_model(images, device='cpu', imgsz=[640,640], conf=0.6, iou=0.5,
                        save=True, save_crop =True, save_txt=True,
                        name=f"{run_name}/", project=f"{project_name}", exist_ok=True)

# Access specific metrics
# print(results.results_dict)
print(f"Done. Processed {len(results)} images") 


Loading the model for testing from meterreader_YOLO/counter-1/weights/best.pt

image 1/240 /Users/yonz/Workspace/images/counter_original/IMG_6900.jpg: 640x448 1 counter, 31.7ms
image 2/240 /Users/yonz/Workspace/images/counter_original/IMG_69002.jpg: 640x448 1 counter, 34.6ms
image 3/240 /Users/yonz/Workspace/images/counter_original/IMG_69003.jpg: 640x448 1 counter, 28.9ms
image 4/240 /Users/yonz/Workspace/images/counter_original/IMG_69004.jpg: 640x448 1 counter, 30.9ms
image 5/240 /Users/yonz/Workspace/images/counter_original/IMG_6901.jpg: 640x448 1 counter, 31.5ms
image 6/240 /Users/yonz/Workspace/images/counter_original/IMG_69012.jpg: 640x448 1 counter, 33.1ms
image 7/240 /Users/yonz/Workspace/images/counter_original/IMG_69013.jpg: 640x448 1 counter, 31.0ms
image 8/240 /Users/yonz/Workspace/images/counter_original/IMG_69014.jpg: 640x448 1 counter, 59.5ms
image 9/240 /Users/yonz/Workspace/images/counter_original/IMG_6902.jpg: 640x512 1 counter, 31.2ms
image 10/240 /Users/yonz/Workspac